In [1]:
# Importing necessary libraries for machine learning and deep learning
import transformers  # Hugging Face library for working with pre-trained models
import torch        # PyTorch library for tensor computations and neural networks
import typing       # Library for type hinting and type annotations

class MedicalChatbot:
    def __init__(self, model_id="aaditya/OpenBioLLM-Llama3-70B"):
        """
        Initialize the medical chatbot with a specific medical language model
        
        Args:
            model_id (str): Identifier for the pre-trained medical language model
        """
        # Create a text generation pipeline using the specified model
        self.pipeline = transformers.pipeline(
            "text-generation",            # Specify the task as text generation
            model=model_id,               # Use the specified model from Hugging Face
            model_kwargs={"torch_dtype": torch.bfloat16},  # Use lower precision for memory efficiency
            device="auto",                # Automatically select best available device (CPU/GPU)
        )
        
        # Define a comprehensive system prompt that sets the context for medical interactions
        self.system_prompt = (
            "You are an expert and experienced medical professional "
            "from the healthcare and biomedical domain with extensive "
            "medical knowledge. Your name is OpenBioLLM, developed by "
            "Saama AI Labs. Provide precise, evidence-based medical "
            "explanations that are scientifically accurate and "
            "comprehensible to a general audience."
        )

    def generate_response(
        self, 
        user_query: str,           # Type hint for user's input query
        max_tokens: int = 256,     # Default maximum response length
        temperature: float = 0.1   # Default temperature for response variability
    ) -> str:                      # Type hint indicating return is a string
        """
        Generate a medical response to a user's query
        
        Args:
            user_query (str): Medical question or prompt
            max_tokens (int): Maximum response length
            temperature (float): Controls response randomness/creativity
        
        Returns:
            str: Generated medical response
        """
        # Construct message list with system and user roles
        messages = [
            {"role": "system", "content": self.system_prompt},  # Set AI's professional context
            {"role": "user", "content": user_query}             # Add user's specific query
        ]
        
        # Apply chat template to format messages for the model
        prompt = self.pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,               # Return as string, not tokens
            add_generation_prompt=True    # Add markers for response generation
        )
        
        # Define token IDs to terminate generation
        terminators = [
            self.pipeline.tokenizer.eos_token_id,                 # Standard end-of-sequence token
            self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Custom end token
        ]
        
        # Generate response using the language model
        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,    # Limit response length
            eos_token_id=terminators,     # Use defined termination tokens
            do_sample=True,               # Enable probabilistic sampling
            temperature=temperature,       # Control response randomness
            top_p=0.9,                    # Nucleus sampling parameter
        )
        
        # Extract and clean the generated response
        return outputs[0]["generated_text"][len(prompt):].strip()



In [5]:
# List of predefined medical questions for testing the model catogorized with dictionaries 
medical_questions = {
    "Basic Medical Knowledge": [
        "What are the primary symptoms of type 2 diabetes?",
        "Explain the pathophysiology of hypertension.",
        "What are the recommended screening protocols for breast cancer?"
    ],
    "Pharmacological Queries": [
        "What are the potential side effects of statins?",
        "How do ACE inhibitors work to manage blood pressure?",
        "Compare the mechanisms of different antidepressant classes."
    ],
    "Diagnostic Reasoning": [
        "What diagnostic tests would you recommend for suspected rheumatoid arthritis?",
        "Describe the differential diagnosis for chest pain in a 45-year-old male."
    ],
    "Treatment & Management": [
        "What are current guidelines for managing type 1 diabetes in adolescents?",
        "Explain the stages of cancer treatment and potential therapies."
    ],
    "Specialized Medical Topics": [
        "How does CRISPR technology potentially impact genetic disease treatment?",
        "What are the latest advances in immunotherapy for cancer?"
    ],
    "Preventive Medicine": [
        "What lifestyle modifications can reduce the risk of cardiovascular disease?",
        "Discuss the importance of vaccination in preventing infectious diseases."
    ],
    "Specialized Medical Scenarios": [
        "What are the complications of untreated gestational diabetes?",
        "Explain the neurological manifestations of multiple sclerosis."
    ]
}

med_cat = "Basic Medical Knowledge" #select the medical category for the user query
User_questions = medical_questions[med_cat] #set the users questions to the selected category


In [6]:
# Main function to demonstrate the chatbot's functionality
def main():
    # Create an instance of the MedicalChatbot
    chatbot = MedicalChatbot()
    
    # Iterate through a subset of medical questions
    for question in User_questions:  # Test the selected questions
        # Print the current query
        print(f"\n🩺 Query: {question}")
        
        # Generate and print the response
        response = chatbot.generate_response(question)
        print(f"📝 Response: {response}")
        
        # Print a separator for readability
        print("-" * 50)



In [ ]:
# Ensure the main function only runs if the script is executed directly
if __name__ == "__main__":
    main()